초기 설정을 수행합니다.

In [ ]:
%pip install boto3 botocore
%pip install opensearch-py
%pip install tqdm
%pip install pdfplumber
%pip install ipywidgets

In [ ]:
%load_ext autoreload
%autoreload 2

1-1. Bedrock Client 를 생성합니다.

In [ ]:
import json
import boto3
import botocore
retry_config = botocore.config.Config(
    retries={"max_attempts": 10, "mode": "standard"}
)
bedrock_client = boto3.Session(
    region_name='us-west-2',
    
).client("bedrock-runtime", config=retry_config)

1-2. OpenSearch 클라이언트를 생성합니다.

In [ ]:
import os
from utilities import get_parameter

In [ ]:
import boto3
region = boto3.Session(region_name='us-west-2').region_name
ssm = boto3.client('ssm', region)

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_user_id',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_user_password',
)

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection

http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
os_client = OpenSearch(
    hosts=[
        {
            'host': opensearch_domain_endpoint.replace("https://", ""),
            'port': 443
        }
    ],
    http_auth=http_auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

2. 문서 전처리

- 문서에 대한 전처리를 진행합니다.
- 주어진 문서의 길이가 너무 길어 Whole Context로 전달할 수 없어, 일정 단위(`document_length`)로 잘라 하나의 Document로 정의합니다.
- 하나의 Document는 다시 일정 단위(`chunk_size`)로 잘라 목록화 합니다. 이를 Chunking 작업이라 합니다.
- 각 Chunk에 대해 Contextual Text를 추가하는 작업을 Situation 이라 합니다.
- Situated Chunk를 대상으로 Embedding이 진행됩니다.

In [ ]:
# Input file and chunking parameters
input_file = 'data/bedrock-ug.pdf'
chunk_size = 1000
start_page = 15
document_length = 20000  # Maximum document size for context

# Extract document name from file path
from pathlib import Path
document_name = Path(input_file).resolve().stem
print(f"Processing document: {document_name}")

In [ ]:
# Import DocumentParser from local library
from document_parser import DocumentParser

# Create output directory if it doesn't exist
os.makedirs("output", exist_ok=True)

try:
    # Load PDF and split into chunks
    print(f"Loading PDF from {input_file} starting at page {start_page}...")
    chunked_document = DocumentParser.split(
        full_text=DocumentParser.load_pdf(input_file, start_page=start_page), 
        chunk_size=chunk_size, 
        max_document_length=document_length
    )
    
    # Define output file path with proper f-string syntax
    output_file = f"output/{document_name}_{chunk_size}_chunks.json"
    
    # Save chunks to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(chunked_document, f, ensure_ascii=False, indent=2)
        print(f"✅ Chunks saved to {output_file}")
        
    # Print summary statistics
    total_chunks = sum(len(doc.get('chunks', [])) for doc in chunked_document)
    print(f"Total documents: {len(chunked_document)}")
    print(f"Total chunks: {total_chunks}")
    
except Exception as e:
    print(f"❌ Error processing document: {str(e)}")

In [ ]:
chunked_document

- 전체 문서에 대한 Context를 바탕으로, 주어진 Chunk의 내용을 5줄 이내의 문장으로 설명하도록 합니다.
- 추가된 문서의 Context는 content 필드에 추가되어 저장됩니다.

In [ ]:
# 170개 Chunk 기준 2분 가량의 시간이 필요합니다.
import time
from tqdm.notebook import tqdm

sys_prompt = """
You're an expert at providing a succinct context, targeted for specific text chunks.

<instruction>
- Offer 1-5 short sentences that explain what specific information this chunk provides within the document.
- Focus on the unique content of this chunk, avoiding general statements about the overall document.
- Clarify how this chunk's content relates to other parts of the document and its role in the document.
- If there's essential information in the document that backs up this chunk's key points, mention the details.
</instruction>
"""
for doc_index, document in enumerate(chunked_document):
    doc_content = document['content']
   
    for chunk in tqdm(document['chunks']):
        # 재시도에 대한 멱등성을 확보하기 위해, 이미 Situate 작업이 완료된 Chunk는 Skip 합니다.
        if chunk['content'].startswith('Context:'):
            continue
            
        document_context_prompt = f"""
        <document>
        {doc_content}
        </document>
        """

        chunk_content = chunk['content']
        chunk_context_prompt = f"""
        Here is the chunk we want to situate within the whole document:

        <chunk>
        {chunk_content}
        </chunk>

        Skip the preamble and only provide the consise context.
        """
        usr_prompt = [{
                "role": "user", 
                "content": [
                    {"text": document_context_prompt},
                    {"text": chunk_context_prompt}
                ]
            }]
        
        try:
            response = bedrock_client.converse(
                modelId='us.anthropic.claude-3-haiku-20240307-v1:0',
                messages=usr_prompt,
                system=[{'text': sys_prompt}],
                inferenceConfig={
                    'maxTokens': 4096
                }
            )

            situated_context = response['output']['message']['content'][0]['text'].strip()
            chunk['content'] = f"Context:\n{situated_context}\n\nChunk:\n{chunk['content']}"
            time.sleep(2)

        except Exception as e:
            print(f"Error generating context for chunk: {e}")



In [ ]:
chunked_document[0]['chunks'][0]

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [ ]:
index_name = 'idx-genai-contextual-retriever'
delete_index_if_exists = True

index_body = {
    "settings": {
        "index.knn": True,
        "index.knn.algo_param.ef_search": 512
    },
    "mappings": {
        "properties": {
            "metadata": {
                "properties": {
                    'parent_id' : {'type': 'keyword'},  
                    'family_tree': {'type': 'keyword'},
                }
            },
            "text": {
                "type": "text",
                "analyzer": "standard"
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {
                    "engine": "faiss",
                    "name": "hnsw",
                    "parameters": {
                        "ef_construction": 512,
                        "m": 16
                    },
                    "space_type": "l2"
                }
            }
        }
    }
}



In [ ]:
if delete_index_if_exists:
    if os_client.indices.exists(index=index_name):
        os_client.indices.delete(index=index_name)
    

os_client.indices.create(index=index_name, body=index_body)


Context가 추가된 Chunk를 Embedding 하고, Opensearch Index에 저장합니다.

In [ ]:
for doc_id, document in enumerate(chunked_document):
    context = chunk['content']
    parent_chunk = {
        'metadata': {
            'family_tree': 'parent',
            'parent_id': None,
            'seq_num': doc_id
        },
        'text': document['content']
    }
    parent_id = os_client.index(
        index=index_name,
        body=parent_chunk #embedded_chunk
    )['_id']

    for chunk_id, chunk in enumerate(tqdm(document['chunks'])):
        context = chunk['content']
        
        response = bedrock_client.invoke_model(
            modelId="amazon.titan-embed-text-v2:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({"inputText": context})
        )
        chunk_embedding = json.loads(response['body'].read())['embedding']

        if chunk_embedding:
            embedded_chunk = {
                "metadata": {
                    "family_tree": "child",
                    "parent_id": parent_id,
                    "seq_num": chunk_id,
                },
                "text": chunk['content'],
                "vector_field": chunk_embedding
            }

        os_client.index(
            index=index_name,
            body=embedded_chunk
        )

        
print(f"Successfully embedded and stored documents in index '{index_name}'")

검색 테스트

Lexical Search (BM-25), Semantic Search (knn)

In [ ]:
from typing import List, Dict

def _format_search_result(hit: Dict, search_method: str) -> Dict:
    print (hit)
    
    return {
        "text": hit['_source']["text"],
        "score": hit['_score'],
        "metadata": hit['_source']['metadata'],
        "search_method": search_method
    }


def search_by_knn(os_client, vector: List[float], index_name: str, top_n: int = 80) -> List[Dict]:
    query = {
        "size": top_n,
        "_source": ["text", "metadata"],
        "query": {
            "knn": {
                "vector_field": {
                    "vector": vector,
                    "k": top_n
                }
            }
        }
    }


    response = os_client.search(index=index_name, body=query)
    return [_format_search_result(hit, 'knn') 
           for hit in response['hits']['hits']]

def search_by_bm25(os_client, query_text: str, index_name: str, top_n: int = 80) -> List[Dict]:
    query = {
        "size": top_n,
        "_source": ["text", "metadata"],
        "query": {
            "match": {
                "text": {
                    "query": query_text,
                    "operator": "or"
                }
            }
        }
    }

    response = os_client.search(index=index_name, body=query)
    return [_format_search_result(hit, 'bm25') 
           for hit in response['hits']['hits']]
    

In [ ]:
def rag(question, index_name, os_client, bedrock_client, emb_model_id="amazon.titan-embed-text-v2:0", search_model_id='anthropic.claude-3-haiku-20240307-v1:0'):
    embedding_response = bedrock_client.invoke_model(
        modelId=emb_model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps({"inputText": question})
    )
    embedding = json.loads(embedding_response['body'].read())['embedding']
    search_results = search_by_knn(os_client, embedding, index_name, 5)
    
    docs = ""
    for result in search_results:
        docs += f"- {result['text']}\n\n"
    
    messages = [{
        'role': 'user',
        'content': [{'text': f"{question}\n\nAdditional Information:\n{docs}"}]
    }]
    
    system_prompt = "You are a helpful AI assistant that provides accurate and concise information about given context."
    
    response = bedrock_client.converse(
        modelId=search_model_id,
        messages=messages,
        system=[{'text': system_prompt}],
        inferenceConfig={
            'maxTokens': 4096
        }
    )
    return response['output']['message']['content'][0]['text']

In [ ]:
index_name = 'idx-genai-contextual-retriever'

In [ ]:
generated = rag('what is knox?', index_name, os_client, bedrock_client)
generated

In [ ]:
# TODO: Hybrid Search + Reranker를 통한 Rank Fusion 구현하기
from reranker_service import RerankerService
def hybrid_rag():
    pass

전체 Document를 가져와서 Q&A 데이터 만들기
- Tool Use를 통해 Q&A 데이터를 생성합니다.

In [ ]:
sys_template = {
    "complex": """
        You are an expert at generating practical questions based on given documentation.
        Your task is to generate complex, reasoning questions and answers.

        Follow these rules:
        1. Generate questions that reflect real user information needs related to the document's subject matter (e.g., technical docs : feature availability, implementation details)
        2. Ensure questions are relevant, concise, preferably under 25 words, and fully answerable with the provided information
        3. Focus on extracting key information that users are likely to seek, while avoiding narrow or less important questions.
        4. When provided with code blocks, focus on understanding the overall functionality rather than the specific syntax or variables. Feel free to request examples of how to use key APIs or features.
        5. Do not use phrases like 'based on the provided context' or 'according to the context'.
    """,
    "simple": """
        You are an expert at generating practical questions based on given documentation.
        Your task is to create simple, directly answerable questions from the given context.

        Follow these rules:
        1. Generate questions that reflect real user information needs related to the document's subject matter (e.g., technical docs : feature availability, implementation details)
        2. Ensure questions are relevant, concise, preferably under 10 words, and fully answerable with the provided information
        3. Focus on extracting key information that users are likely to seek, while avoiding narrow or less important questions.
        4. When provided with code blocks, focus on understanding the overall functionality rather than the specific syntax or variables. Feel free to request examples of how to use key APIs or features.
        5. Do not use phrases like 'based on the provided context' or 'according to the context'.
    """
}

In [ ]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "QuestionAnswerGenerator",
                "description": "Generates questions and answers based on the given context.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "question": {
                                "type": "string",
                                "description": "The generated question"
                            },
                            "answer": {
                                "type": "string",
                                "description": "The answer to the generated question"
                            }
                        },
                        "required": ["question", "answer"]
                    }
                }
            }
        }
    ]
}

In [ ]:
import random
import json
import uuid
from tqdm.notebook import tqdm

context = '\n\n'.join([doc.get('content', '') for doc in chunked_document])
qa_dataset = []

generated_question = {"simple": [], "complex": []}

for i in tqdm(range(4)):
    if i % 2 == 0:
        question_type = "complex"
    else:
        question_type = "simple"

    user_template = f"""
    Generate a {question_type} question and its answer based on the following context:

    Context: {context}

    Use the QuestionAnswerGenerator tool to provide the output.
    """

    sys_prompt = [{"text": sys_template[question_type]}]
    user_prompt = [{"role": "user", "content": [{"text": user_template}]}]
    temperature = 0.0
    top_p = 0.5
    inference_config = {"temperature": temperature, "topP": top_p}

    response = bedrock_client.converse(
        modelId='anthropic.claude-3-5-sonnet-20240620-v1:0',
        messages=user_prompt,
        toolConfig=tool_config,
        system=sys_prompt,
        inferenceConfig={
            'maxTokens': 4096
        }
    )

    stop_reason = response['stopReason']

    if stop_reason == 'tool_use':
        tool_requests = response['output']['message']['content']

        for tool_request in [x for x in tool_requests if 'toolUse' in x]:
            if tool_request['toolUse']['name'] == 'QuestionAnswerGenerator':
                res = tool_request['toolUse']['input']

                qa_item = {
                    "question": tool_request['toolUse']['input']['question'],
                    "ground_truth": tool_request['toolUse']['input']['answer'],
                    "question_type": question_type,
                    # "context": context
                }
                
                qa_dataset.append(qa_item)


In [ ]:
qa_dataset

In [ ]:
answer = rag(qa_dataset[0]['question'], index_name, os_client, bedrock_client)
answer

테스트 Q&A 데이터 셋을 활용하여, Contextual RAG의 답변의 유사도를 평가합니다.

In [ ]:
evaluate_system_prompt = """
Evaluate the correctness of the generation on a continuous scale from 0 to 1. A generation can be considered correct (Score: 1) if it includes all the key facts from the ground truth and if every fact presented in the generation is factually supported by the ground truth or common sense.
Example:
Query: Can eating carrots improve your vision?
Answer: Yes, eating carrots significantly improves your vision, especially at night. This is why people who eat lots of carrots never need glasses. Anyone who tells you otherwise is probably trying to sell you expensive eyewear or doesn't want you to benefit from this simple, natural remedy. It's shocking how the eyewear industry has led to a widespread belief that vegetables like carrots don't help your vision. People are so gullible to fall for these money-making schemes.
Ground truth: Well, yes and no. Carrots won’t improve your visual acuity if you have less than perfect vision. A diet of carrots won’t give a blind person 20/20 vision. But, the vitamins found in the vegetable can help promote overall eye health. Carrots contain beta-carotene, a substance that the body converts to vitamin A, an important nutrient for eye health. An extreme lack of vitamin A can cause blindness. Vitamin A can prevent the formation of cataracts and macular degeneration, the world’s leading cause of blindness. However, if your vision problems aren’t related to vitamin A, your vision won’t change no matter how many carrots you eat.
Score: 0.1
Reasoning: While the generation mentions that carrots can improve vision, it fails to outline the reason for this phenomenon and the circumstances under which this is the case. The rest of the response contains misinformation and exaggerations regarding the benefits of eating carrots for vision improvement. It deviates significantly from the more accurate and nuanced explanation provided in the ground truth.
"""

eval_tools = {
    "tools": [
        {
            "toolSpec": {
                "name": "CorrectressGrader",
                "description": "Evaluate the correctness of the answer on a continuous scale from 0 to 1, and reasoning why the score is. A generation can be considered correct (Score: 1) if it includes all the key facts from the ground truth and if every fact presented in the generation is factually supported by the ground truth.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "score": {
                                "type": "number",
                                "description": "The correctress score [0.0, 1.0]"
                            },
                            "reason": {
                                "type": "string",
                                "description": "The reason about the score"
                            }
                        },
                        "required": ["score", "reason"]
                    }
                }
            }
        }
    ]
}


In [ ]:
from tqdm.notebook import tqdm

results = []

for question_data in tqdm(qa_dataset):
    question = question_data['question']
    ground_truth = question_data['ground_truth']

    generated = rag(question=question, index_name=index_name, os_client=os_client, bedrock_client=bedrock_client)
    
    evaluate_user_template = f"""
    Query: {question}
    Answer: {generated}
    Ground Truth: {ground_truth}
    """

    sys_prompt = [{"text": evaluate_system_prompt}]
    user_prompt = [{"role": "user", "content": [{"text": evaluate_user_template}]}]
    temperature = 0.0
    top_p = 0.5
    inference_config = {"temperature": temperature, "topP": top_p}

    response = bedrock_client.converse(
        modelId='anthropic.claude-3-5-sonnet-20240620-v1:0',
        messages=user_prompt,
        toolConfig=eval_tools,
        system=sys_prompt,
        inferenceConfig={
            'maxTokens': 4096
        }
    )
    
    stop_reason = response['stopReason']

    if stop_reason == 'tool_use':
        tool_requests = response['output']['message']['content']
        

        for tool_request in [x for x in tool_requests if 'toolUse' in x]:
            if tool_request['toolUse']['name'] == 'CorrectressGrader':
                res = tool_request['toolUse']['input']

                result = {
                     "question": question,
                     "question_type": question_data['question_type'],
                     "generated_answer": generated,
                     "ground_truth": ground_truth,
                     "score": res['score']
                }

                results.append(result)

results